## 0. Requirements

To run this notebook, start jupyter notebook with the ae483 environment active (see [MacOS](https://canvas.illinois.edu/courses/7116/modules/items/1230512) or [Windows](https://canvas.illinois.edu/courses/7116/modules/items/1270318) instructions for how to create this environment).

## 1. Set up the notebook

Import all the modules we need.

In [ ]:
import numpy as np
import sympy as sym

## 2. Show how to take cross products

### Taking cross products with numpy

Suppose we want to take the cross product
$ w \times v $
where $w$ and $v$ are described in coordinates as follows:

In [ ]:
w_in1 = np.array([1., 2., 3.])
v_in1 = np.array([4., 5., 6.])

The easiest way to do this is with [numpy.cross](https://numpy.org/doc/stable/reference/generated/numpy.cross.html):

In [ ]:
print(np.cross(w_in1, v_in1))

An alternative is to use the wedge operator. Here is a function that implements this operator:

In [ ]:
def wedge_numpy(v):
    """
    Returns a skew-symmetric matrix M that can be used to implement
    the cross product "v x w" as the matrix product "M @ w" - assumes
    that v has shape (3, )
    """
    return np.array([[0., -v[2], v[1]], [v[2], 0., -v[0]], [-v[1], v[0], 0.]])

Here is how we would use the wedge operator to take the same cross product as before:

In [ ]:
print(wedge_numpy(w_in1) @ v_in1)

### Taking cross products with sympy

Suppose we want to take the cross product
$ w \times v $
where $w$ and $v$ are described in coordinates as follows:

$$w^1 = \begin{bmatrix} a \\ b \\ c \end{bmatrix} \qquad\qquad v^1 = \begin{bmatrix} d \\ e \\ f \end{bmatrix}$$

First, create symbolic variables.

In [ ]:
a, b, c, d, e, f = sym.symbols('a, b, c, d, e, f')

Second, create $w$ and $v$ as sympy matrices:

In [ ]:
w_in1 = sym.Matrix([a, b, c])
v_in1 = sym.Matrix([d, e, f])

Now, the easiest way to take the cross product is with [sympy.cross](https://docs.sympy.org/latest/modules/matrices/matrices.html#sympy.matrices.matrices.MatrixBase.cross):

In [ ]:
w_in1.cross(v_in1)

An alternative is to use the wedge operator. Here is a function that implements this operator:

In [ ]:
def wedge_sympy(v):
    """
    Returns a skew-symmetric matrix M that can be used to implement
    the cross product "v x w" as the matrix product "M @ w" - assumes
    that v has shape (3, )
    """
    return sym.Matrix([[0., -v[2], v[1]], [v[2], 0., -v[0]], [-v[1], v[0], 0.]])

Here is how we would use the wedge operator to take the same cross product as before:

In [ ]:
wedge_sympy(w_in1) * v_in1

Let's plug in the same values we used for the numpy example, to confirm we get the same result:

In [ ]:
w_in1.cross(v_in1).subs([(a, 1), (b, 2), (c, 3), (d, 4), (e, 5), (f, 6)])

## 3. Derive relationship between rotor speeds and forces and torques

Define parameters as symbolic variables.

In [ ]:
k_F, k_M, l = sym.symbols('k_F, k_M, l')

Define rotor speeds as symbolic variables.

In [ ]:
sigma_1, sigma_2, sigma_3, sigma_4 = sym.symbols('sigma_1, sigma_2, sigma_3, sigma_4')

Find net force and torque due to the first motor (as an example).

In [ ]:
# Net force
f_1_in1 = sym.Matrix([0, 0, k_F * sigma_1**2])

# Point at which force is applied
p_1_in1 = sym.Matrix([l, -l, 0])

# Net torque
tau_1_in1 = sym.Matrix([0, 0, -k_M * sigma_1**2]) + p_1_in1.cross(f_1_in1)

Show the result.

In [ ]:
tau_1_in1

Define the matrix $P$ that maps rotor speeds to net forces and torques:

$$
\begin{bmatrix} \tau_x \\ \tau_y \\ \tau_z \\ f_z \end{bmatrix}
=
P
\begin{bmatrix} \sigma_1^2 \\ \sigma_2^2 \\ \sigma_3^2 \\ \sigma_4^2 \end{bmatrix}
$$

In [ ]:
P = sym.Matrix([[ -l * k_F, -l * k_F,  l * k_F,  l * k_F  ],
                [ -l * k_F, l * k_F,   l * k_F,  -l * k_F ],
                [ -k_M,     k_M,       -k_M,     k_M      ],
                [ k_F,      k_F,       k_F,      k_F      ]])

Show the result.

In [ ]:
P

Find the matrix $P^{-1}$ that maps forces and torques to rotor speeds:

$$
\begin{bmatrix} \sigma_1^2 \\ \sigma_2^2 \\ \sigma_3^2 \\ \sigma_4^2 \end{bmatrix}
=
P^{-1}
\begin{bmatrix} \tau_x \\ \tau_y \\ \tau_z \\ f_z \end{bmatrix}
$$

In [ ]:
Pinv = P.inv()

Show the result.

In [ ]:
Pinv

This result can be easier to visualize when you factor out the denominator in each term:

In [ ]:
sym.simplify(Pinv * 4 * k_F * k_M * l)